In [8]:
import pandas as pd
from datetime import date, timedelta

work_dir = '/home/jovyan/work/' #GPU server directory Duke Scavenger

In [ ]:
#Download User Session Log dataset from the AI Crowd website page of Spotify Skip Prediction challenge
!wget -c https://os.zhdk.cloud.switch.ch/swift/v1/crowdai-public/spotify-sequential-skip-prediction-challenge/20181113_training_set.tar.gz

In [ ]:
#Extract the dataset into files
!tar xvzf 20181113_training_set.tar.gz

In [ ]:
#Download Songs dataset from the AI Crowd website page of Spotify Skip Prediction challenge
!wget -c https://os.zhdk.cloud.switch.ch/swift/v1/crowdai-public/spotify-sequential-skip-prediction-challenge/20181120_track_features.tar.gz

In [ ]:
!tar xvzf 20181120_track_features.tar.gz

In [6]:
!ls -ltr training_set/*20180720*.csv

ls: cannot access 'training_set/*20180720*.csv': No such file or directory


!ls -ltr track_features/*.csv

## Create Dataset from 15th July to 10th August

In [ ]:
start_date = date(2018, 7, 15)
end_date = date(2018, 8, 10)
delta = timedelta(days=1)
out = pd.DataFrame(columns=['track_id', 'skip_2', 'premium', 'hist_user_behavior_is_shuffle', 'context_type', 'total_count'])

while start_date <= end_date:
    current_date = start_date.strftime("%Y%m%d")
    start_date += delta
        
    #print(current_date + ' :start')
    #For each date, filter and merge the data from all the files and join with track features
    
    for i in range(0,10):
        #print('\t \t' + str(i) + ' :start')
        df = pd.read_csv(work_dir+'training_set/log_'+str(i)+'_'+current_date+'_000000000000.csv')
        df = df.rename({'track_id_clean': 'track_id'}, axis=1)
        df = df[['track_id','skip_2', 'premium', 'hist_user_behavior_is_shuffle', 'context_type']]

        df['skip_2'] = (df['skip_2'] == True ).astype(int)
        df['premium'] = (df['premium'] == True ).astype(int)
        df['hist_user_behavior_is_shuffle'] = (df['hist_user_behavior_is_shuffle'] == True ).astype(int)
        
        df.context_type.replace(to_replace=dict(editorial_playlist=0, user_collection=1, radio=2,personalized_playlist=3, catalog=4, charts=5), inplace=True)
        
        grouped = df.groupby(['track_id', 'skip_2', 'premium', 'hist_user_behavior_is_shuffle', 'context_type']).size().reset_index()
        grouped = grouped.rename({0: 'total_count'}, axis=1)
        
        out = pd.merge(grouped, out, how='outer', on=['track_id', 'skip_2','premium', 'hist_user_behavior_is_shuffle', 'context_type'])\
        .set_index(['track_id','skip_2','premium', 'hist_user_behavior_is_shuffle', 'context_type'])\
        .sum(axis=1).reset_index()
        
        out = out.rename({0: 'count'}, axis=1)
        #print('\t \t' + str(i) + ' :end')
        pass
    print(sum(out['count']))
    print(current_date + ' :end')
    pass

out.to_csv(work_dir+'agg2018'+'.csv', header=True, sep=',') #Save file for backup so do not have to process this chunk again

In [ ]:
%reset_selective df
%reset_selective grouped

# Import Sound track dataset and merge the files into one dataframe

In [ ]:
#Import and combine track features in a dataframe
track0 = pd.read_csv(work_dir + 'track_features/tf_000000000000.csv')
print(len(track0))

In [ ]:
track1 = pd.read_csv(work_dir + 'track_features/tf_000000000001.csv')
print(len(track1))

In [ ]:
#Remove records from 1950s to 1970s as the songs are heard by a very group of people
track0 = track0[track0['release_year']>1970]
print(len(track0))

track1 = track1[track1['release_year']>1970]
print(len(track1))

In [ ]:
track_features = track0.append(track1, ignore_index=False, sort=False)
#track_features['track_id'] = track_features['track_id'].astype(str)
print(len(track_features))

#Round all decimal values upto 4 decimal to decrease the dataset size
track_features = track_features.round(decimals=4)

track_features['mode'].replace(to_replace=dict(minor=0, major=1), inplace=True)

In [ ]:
%reset_selective track0
%reset_selective track1

# Join Tracks with User Session Data

In [ ]:
#out = pd.read_csv(work_dir+'agg2018.csv')

jn = out.set_index('track_id').join(track_features.set_index('track_id'), on='track_id', how='inner').reset_index()

jn.to_csv(work_dir+'final_dataset'+'.csv', header=True, sep=',')

In [ ]:
%reset_selective out

In [ ]:
#jn = pd.read_csv(work_dir+'final_dataset'+'.csv')
print(jn.columns)
jn = jn[['track_id', 'skip_2','premium', 'hist_user_behavior_is_shuffle', 'context_type',
       'duration', 'release_year', 'us_popularity_estimate', 'acousticness',
       'beat_strength', 'bounciness', 'danceability',
       'energy', 'flatness', 'instrumentalness', 'liveness', 'loudness',
       'mechanism', 'organism', 'speechiness', 'tempo', 'valence', 'count']]

In [ ]:
count = jn.groupby(['track_id'])['count'].sum().reset_index()
count = count.rename({'count': 'total_count'}, axis=1)

In [ ]:
join = jn.set_index('track_id').join(count.set_index('track_id'), on='track_id', how='inner').reset_index()
join.shape

In [ ]:
join.to_csv(work_dir+'final_dataset_cleaned'+'.csv', header=True, sep=',')

In [ ]:
%reset_selective count

In [ ]:
#Remove all songs which have been heard less than 50 times in a month as they will stand out as outliers
join = join[join['total_count']>49]

In [ ]:
#All the sound tracks which have been skipped and find count of those records
skipped = join[join['skip_2']==1]
skipped_count = skipped.groupby(['track_id', 'premium', 'hist_user_behavior_is_shuffle',
       'context_type', 'duration', 'release_year', 'us_popularity_estimate',
       'acousticness', 'beat_strength', 'bounciness', 'danceability', 'energy',
       'flatness', 'instrumentalness', 'liveness', 'loudness', 'mechanism',
       'organism', 'speechiness', 'tempo', 'valence'])['count'].sum().reset_index()
skipped_count= skipped_count.rename({'count': 'skipped_count'}, axis=1)

In [ ]:
#All the sound tracks which have been NOT been skipped and find count of those records
not_skipped = join[join['skip_2']==0]
not_skipped_count = not_skipped.groupby(['track_id', 'premium', 'hist_user_behavior_is_shuffle',
       'context_type', 'duration', 'release_year', 'us_popularity_estimate',
       'acousticness', 'beat_strength', 'bounciness', 'danceability', 'energy',
       'flatness', 'instrumentalness', 'liveness', 'loudness', 'mechanism',
       'organism', 'speechiness', 'tempo', 'valence'])['count'].sum().reset_index()
not_skipped_count= not_skipped_count.rename({'count': 'not_skipped_count'}, axis=1)

In [ ]:
%reset_selective join

In [ ]:
joined_counts = skipped_count.set_index(['track_id', 'premium', 'hist_user_behavior_is_shuffle',
       'context_type', 'duration', 'release_year', 'us_popularity_estimate',
       'acousticness', 'beat_strength', 'bounciness', 'danceability', 'energy',
       'flatness', 'instrumentalness', 'liveness', 'loudness', 'mechanism',
       'organism', 'speechiness', 'tempo', 'valence']).join(not_skipped_count.set_index(['track_id', 'premium', 'hist_user_behavior_is_shuffle',
       'context_type', 'duration', 'release_year', 'us_popularity_estimate',
       'acousticness', 'beat_strength', 'bounciness', 'danceability', 'energy',
       'flatness', 'instrumentalness', 'liveness', 'loudness', 'mechanism',
       'organism', 'speechiness', 'tempo', 'valence']), on=['track_id', 'premium', 'hist_user_behavior_is_shuffle',
       'context_type', 'duration', 'release_year', 'us_popularity_estimate',
       'acousticness', 'beat_strength', 'bounciness', 'danceability', 'energy',
       'flatness', 'instrumentalness', 'liveness', 'loudness', 'mechanism',
       'organism', 'speechiness', 'tempo', 'valence'], how='outer').reset_index()

In [ ]:
%reset_selective skipped_count
%reset_selective not_skipped_count

In [ ]:
joined_counts.fillna(0,inplace=True)
joined_counts['total_count']=joined_counts['skipped_count']+joined_counts['not_skipped_count']

joined_counts.to_csv('final_agg.csv', header=True, sep=',')

## Data Transformation

In [9]:
joined_counts['release_year_tr'] = '70s'
joined_counts.loc[joined_counts['release_year']>1979, 'release_year_tr'] = '80s'
joined_counts.loc[joined_counts['release_year']>1989, 'release_year_tr'] = '90s'
joined_counts.loc[joined_counts['release_year']>1999, 'release_year_tr'] = '20s'
joined_counts.loc[joined_counts['release_year']>2010, 'release_year_tr'] = '21s'

NameError: name 'joined_counts' is not defined

In [ ]:
joined_counts.to_csv('final_agg_50_r_yr.csv', header=True, sep=',')